In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:20pt;}
div.text_cell_render.rendered_html{font-size:20pt;}
div.text_cell_render ul li, div.text_cell_render ol li p, code{font-size:22pt; line-height:30px;}
div.output {font-size:24pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:24pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:24pt;padding:5px;}
table.dataframe{font-size:24px;}
</style>
"""))

# <span style='color:red'>ch06_RNN기반의 Seq2Seq(스마트번역기)</span>

 - Google Neutral machine Translation (GNMT)
 - RNN 기반의 Sequence to Sequence 방식
 - 인코더(입력)/디코더(모법출력)연결구조
 * 응용분야: 자연어

# 패키지 및 하이퍼 파라미터

In [2]:
import pandas as pd
import numpy as np
from time import time
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

#하이파라미터
MY_HIDDEN = 128
MY_EPOCH  = 1000

# 학습데이터

In [3]:
raw = pd.read_csv('data/translate.csv',header=None)
eng_kor = raw.values.tolist() #데이터프레임을 list로 변환
print(eng_kor[:3])
print('영-한 번역데이터 갯수: ', len(eng_kor))

[['cold', '감기'], ['come', '오다'], ['cook', '요리']]
영-한 번역데이터 갯수:  110


# 영어 알파벳과 한글 문자 리스트 만들기

In [4]:
e_alpha = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz']
korean = ''.join([data[1] for data in eng_kor])
k_ch = list(set([ch for ch in korean]))
k_ch.sort()
k_alpha = pd.read_csv('data/korean.csv',header=None)[0].tolist()
k_alpha == k_ch #순서와 내용이 같은지 여부

True

In [5]:
from collections import Counter
list1 = ['가','나','다']
list2 = ['나','다','가']
print(list1 == list2)
print(Counter(list1)==Counter(list2))

False
True


In [6]:
alpha = e_alpha + k_alpha
print('영어와 한글 알파벳 : ', alpha)
alpha_total_size = len(alpha) # 171(원핫인코딩 사이즈)
print('전체 알파벳 갯수(원핫인코딩 사이즈) : ', alpha_total_size)

영어와 한글 알파벳 :  ['S', 'E', 'P', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '가', '각', '간', '감', '개', '거', '것', '게', '계', '고', '관', '광', '구', '굴', '규', '그', '금', '기', '깊', '나', '날', '남', '내', '넓', '녀', '노', '놀', '농', '높', '뉴', '늦', '다', '단', '도', '동', '들', '람', '랑', '래', '램', '류', '름', '릎', '리', '많', '망', '매', '머', '먼', '멍', '메', '명', '모', '목', '무', '물', '미', '바', '반', '방', '번', '복', '부', '분', '붕', '비', '뿌', '사', '상', '색', '생', '서', '선', '소', '손', '수', '쉽', '스', '시', '식', '실', '싸', '아', '약', '얇', '어', '언', '얼', '여', '연', '오', '옥', '왼', '요', '용', '우', '운', '움', '위', '유', '은', '을', '음', '의', '이', '익', '인', '읽', '입', '자', '작', '장', '적', '제', '좋', '주', '지', '짜', '쪽', '찾', '책', '출', '칙', '크', '키', '탈', '택', '통', '파', '팔', '편', '피', '핑', '한', '합', '해', '행', '험', '회', '획', '휴', '흐']
전체 알파벳 갯수(원핫인코딩 사이즈) :  171


# 문자당 num을 갖는 dict

In [7]:
char_to_num = {}
for i, c in enumerate(alpha):
#     print(i,c)
    char_to_num[c] = i
print(char_to_num)

{'S': 0, 'E': 1, 'P': 2, 'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'j': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'w': 25, 'x': 26, 'y': 27, 'z': 28, '가': 29, '각': 30, '간': 31, '감': 32, '개': 33, '거': 34, '것': 35, '게': 36, '계': 37, '고': 38, '관': 39, '광': 40, '구': 41, '굴': 42, '규': 43, '그': 44, '금': 45, '기': 46, '깊': 47, '나': 48, '날': 49, '남': 50, '내': 51, '넓': 52, '녀': 53, '노': 54, '놀': 55, '농': 56, '높': 57, '뉴': 58, '늦': 59, '다': 60, '단': 61, '도': 62, '동': 63, '들': 64, '람': 65, '랑': 66, '래': 67, '램': 68, '류': 69, '름': 70, '릎': 71, '리': 72, '많': 73, '망': 74, '매': 75, '머': 76, '먼': 77, '멍': 78, '메': 79, '명': 80, '모': 81, '목': 82, '무': 83, '물': 84, '미': 85, '바': 86, '반': 87, '방': 88, '번': 89, '복': 90, '부': 91, '분': 92, '붕': 93, '비': 94, '뿌': 95, '사': 96, '상': 97, '색': 98, '생': 99, '서': 100, '선': 101, '소': 102, '손': 103, '수': 104, '쉽': 105, '스': 106, '시': 107, '식': 108, '실': 109, '싸': 110,

In [8]:
char_to_num = {c:i for i, c in enumerate(alpha)}
print(char_to_num)

{'S': 0, 'E': 1, 'P': 2, 'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'j': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'w': 25, 'x': 26, 'y': 27, 'z': 28, '가': 29, '각': 30, '간': 31, '감': 32, '개': 33, '거': 34, '것': 35, '게': 36, '계': 37, '고': 38, '관': 39, '광': 40, '구': 41, '굴': 42, '규': 43, '그': 44, '금': 45, '기': 46, '깊': 47, '나': 48, '날': 49, '남': 50, '내': 51, '넓': 52, '녀': 53, '노': 54, '놀': 55, '농': 56, '높': 57, '뉴': 58, '늦': 59, '다': 60, '단': 61, '도': 62, '동': 63, '들': 64, '람': 65, '랑': 66, '래': 67, '램': 68, '류': 69, '름': 70, '릎': 71, '리': 72, '많': 73, '망': 74, '매': 75, '머': 76, '먼': 77, '멍': 78, '메': 79, '명': 80, '모': 81, '목': 82, '무': 83, '물': 84, '미': 85, '바': 86, '반': 87, '방': 88, '번': 89, '복': 90, '부': 91, '분': 92, '붕': 93, '비': 94, '뿌': 95, '사': 96, '상': 97, '색': 98, '생': 99, '서': 100, '선': 101, '소': 102, '손': 103, '수': 104, '쉽': 105, '스': 106, '시': 107, '식': 108, '실': 109, '싸': 110,

In [9]:
# 숫자->문자/ 문자->숫자
print('숫자 -> 문자 : ', alpha[5])
print('문자 -> 숫자 : ', char_to_num['c'])


숫자 -> 문자 :  c
문자 -> 숫자 :  5


In [10]:
data = eng_kor[0]
print (data)
#char_to_num['c'], char_to_num['o'], char_to_num['l'], char_to_num['d']
print('인코더 입력 : ', [char_to_num[ch] for ch in data[0]])
print('디코더 입력 : ', [char_to_num[ch] for ch in 'S'+data[1]])
print('디코더 출력 : ', [char_to_num[ch] for ch in data[1]+'E'])

['cold', '감기']
인코더 입력 :  [5, 17, 14, 6]
디코더 입력 :  [0, 32, 46]
디코더 출력 :  [32, 46, 1]


In [11]:
#원핫인코딩방법 1: pd.get_dummies([5,17,14,6]) - 이 코드에서 적용 불가 
pd.get_dummies([5,17,14,6])

,5,6,14,17
0,1,0,0,0
1,0,0,0,1
2,0,0,1,0
3,0,1,0,0


In [12]:
#원핫인코딩 방법2 : to_categorical([5,3,7],num_classes=10)
to_categorical([5,3,7], num_classes=10)

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]], dtype=float32)

In [13]:
#원핫인코딩 방법3 : np.eye(10)
np.eye(10)[[5,3,7]]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]])

# 인코더 입력, 디코더 입력, 디코더 출력
* 인코더 입력(원핫 인코딩 ㅇ), 디코더입력(원핫인코딩ㅇ), 디코더 출력(원핫인코딩x)

In [14]:
def encoding(eng_kor=eng_kor):
    '인코더입력, 디코더입력, 디코더출력 데이터를 return'
    enc_in = [] # 인코더 입력
    dec_in = [] # 디코더 입력
    dec_out = [] # 디코더 출력
    for data in eng_kor:
        # 인코더 입력 데이터 (영어 -> 숫자 -> 원핫인코딩)
        eng = [char_to_num[ch] for ch in data[0]]
        #print(data[0], eng)
        #eng_one = to_categorical(eng, num_classes=alpha_total_size)#alpha_total_size:171
        eng_one = np.eye(alpha_total_size)[eng]
        enc_in.append(eng_one)
        # 디코더 입력 데이터 ('S한글' -> 숫자 -> 원핫인코딩)
        kor = [char_to_num[ch] for ch in 'S'+data[1]]
        kor_one = np.eye(alpha_total_size)[kor]
        dec_in.append(kor_one)
        # 디코더 출력 ('한글E' -> 숫자 )
        kor = [char_to_num[ch] for ch in data[1]+'E']
        dec_out.append(kor)
    # 인공신경에 넣을 데이터이므로 numpy 배열로 전환
    enc_in = np.array(enc_in)
    dec_in = np.array(dec_in)
    dec_out = np.array(dec_out)
    #print(enc_in.shape, dec_in.shape, dec_out.shape)
    return enc_in, dec_in, dec_out

sample = [['wood', '나무'],
          ['word','단어']]

In [15]:
#RNN 분류분석 : 타겟변수를 원핫인코딩(시스템에게 원핫인코딩 의뢰)
X_enc, X_dec, y_dec = encoding(sample)
X_enc.shape, X_dec.shape, y_dec.shape

((2, 4, 171), (2, 3, 171), (2, 3))

In [16]:
y_dec

array([[ 48,  83,   1],
       [ 61, 114,   1]])

# 축증가

In [17]:
y_dec.reshape(2,3,1) #방법1

array([[[ 48],
        [ 83],
        [  1]],

       [[ 61],
        [114],
        [  1]]])

In [18]:
np.expand_dims(y_dec, axis=-1)#방법2

array([[[ 48],
        [ 83],
        [  1]],

       [[ 61],
        [114],
        [  1]]])

In [19]:
y_dec[...,np.newaxis]#방법3

array([[[ 48],
        [ 83],
        [  1]],

       [[ 61],
        [114],
        [  1]]])

In [20]:
y_dec[:,:,None]#방법4

array([[[ 48],
        [ 83],
        [  1]],

       [[ 61],
        [114],
        [  1]]])

# 전체 번역데이터(독립변수, 타겟변수)

In [21]:
X_enc, X_dec, y_dec = encoding(eng_kor)
Y_dec = np.expand_dims(y_dec, axis = -1)
#Y_dec = y_dec[...,np.newaxis]
#Y_dec = y_dec.reshape(-1,3,1)
X_enc.shape, X_dec.shape, y_dec.shape

((110, 4, 171), (110, 3, 171), (110, 3))

# Sequence to Sequence 모델구현

In [22]:
# 인코더 LSTM 구현
ENC_IN = Input(shape=(4, alpha_total_size))
# 윗출력, state_c, state_h
_, state_h, state_c = LSTM(
                    units=MY_HIDDEN,
                    # return_sequences=False 윗출력 안받음
                    return_state=True, 
            )(ENC_IN)
# 인코더와 디코더를 연결할 link
link = [state_h, state_c]
# 디코더 구현 : return_sequences=True 위로 올라는 출력값 사용
DEC_IN = Input(shape=(3, alpha_total_size))
DEC_MID = LSTM(units=MY_HIDDEN,
              # return_state=False
              return_sequences=True)(DEC_IN,
                                    initial_state=link)
# 최종 출력층
DEC_OUT = Dense(units=alpha_total_size,
               activation='softmax')(DEC_MID)

#모델
model = Model(inputs = [ENC_IN, DEC_IN],
             outputs=DEC_OUT)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 4, 171)]     0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 3, 171)]     0           []                               
                                                                                                  
 lstm (LSTM)                    [(None, 128),        153600      ['input_1[0][0]']                
                                 (None, 128),                                                     
                                 (None, 128)]                                                     
                                                                                              

# Attention 모델 구현

In [23]:
from tensorflow.keras.layers import Attention, Concatenate

# 인코더 LSTM 구현
ENC_IN = Input(shape=(4, alpha_total_size))
# 윗출력, state_c, state_h
ENC_OUT, state_h, state_c = LSTM(
                    units=MY_HIDDEN,
                    return_sequences=True,
                    return_state=True, 
            )(ENC_IN)
# 인코더와 디코더를 연결할 link
link = [state_h, state_c]
# 디코더 구현 : return_sequences=True 위로 올라는 출력값 사용
DEC_IN = Input(shape=(3, alpha_total_size))
DEC_MID, _, _ = LSTM(units=MY_HIDDEN,
              return_state=True,
              return_sequences=True)(DEC_IN,
                                    initial_state=link)
# 어텐션 메커니즘
CONTEXT_VECTOR = Attention()([DEC_MID, ENC_OUT])

# 컨텍스트벡터와 디코더LSTM출력을 결합
CONTEXT_AND_LSTM_OUT = Concatenate()([CONTEXT_VECTOR, DEC_MID])

# 최종 출력층
OUT = Dense(units=alpha_total_size,
               activation='softmax')(CONTEXT_AND_LSTM_OUT)
# 모델
model = Model(inputs=[ENC_IN, DEC_IN],
             outputs=OUT)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 4, 171)]     0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 3, 171)]     0           []                               
                                                                                                  
 lstm_2 (LSTM)                  [(None, 4, 128),     153600      ['input_3[0][0]']                
                                 (None, 128),                                                     
                                 (None, 128)]                                                     
                                                                                            

# 학습과정 설정 및 학습하기

In [24]:
model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = 'rmsprop',
              metrics = ['accuracy'])
start = time()
hist = model.fit([X_enc, X_dec], Y_dec,
                epochs = MY_EPOCH,
                verbose=2)
end = time()
print('학습시간 : ',{end-start})

Epoch 1/1000
4/4 - 1s - loss: 5.1183 - accuracy: 0.2030 - 1s/epoch - 292ms/step
Epoch 2/1000
4/4 - 0s - loss: 4.9796 - accuracy: 0.3333 - 17ms/epoch - 4ms/step
Epoch 3/1000
4/4 - 0s - loss: 4.3734 - accuracy: 0.3333 - 16ms/epoch - 4ms/step
Epoch 4/1000
4/4 - 0s - loss: 3.4607 - accuracy: 0.3333 - 16ms/epoch - 4ms/step
Epoch 5/1000
4/4 - 0s - loss: 3.3981 - accuracy: 0.3333 - 16ms/epoch - 4ms/step
Epoch 6/1000
4/4 - 0s - loss: 3.3576 - accuracy: 0.3333 - 17ms/epoch - 4ms/step
Epoch 7/1000
4/4 - 0s - loss: 3.3236 - accuracy: 0.3333 - 16ms/epoch - 4ms/step
Epoch 8/1000
4/4 - 0s - loss: 3.2965 - accuracy: 0.3333 - 16ms/epoch - 4ms/step
Epoch 9/1000
4/4 - 0s - loss: 3.2690 - accuracy: 0.3333 - 15ms/epoch - 4ms/step
Epoch 10/1000
4/4 - 0s - loss: 3.2633 - accuracy: 0.3333 - 14ms/epoch - 4ms/step
Epoch 11/1000
4/4 - 0s - loss: 3.2270 - accuracy: 0.3333 - 15ms/epoch - 4ms/step
Epoch 12/1000
4/4 - 0s - loss: 3.2006 - accuracy: 0.3394 - 16ms/epoch - 4ms/step
Epoch 13/1000
4/4 - 0s - loss: 3.1782

Epoch 103/1000
4/4 - 0s - loss: 0.2092 - accuracy: 0.9909 - 16ms/epoch - 4ms/step
Epoch 104/1000
4/4 - 0s - loss: 0.2031 - accuracy: 0.9879 - 16ms/epoch - 4ms/step
Epoch 105/1000
4/4 - 0s - loss: 0.1917 - accuracy: 0.9879 - 16ms/epoch - 4ms/step
Epoch 106/1000
4/4 - 0s - loss: 0.1863 - accuracy: 0.9818 - 16ms/epoch - 4ms/step
Epoch 107/1000
4/4 - 0s - loss: 0.1802 - accuracy: 0.9848 - 16ms/epoch - 4ms/step
Epoch 108/1000
4/4 - 0s - loss: 0.1660 - accuracy: 0.9848 - 16ms/epoch - 4ms/step
Epoch 109/1000
4/4 - 0s - loss: 0.1610 - accuracy: 0.9818 - 15ms/epoch - 4ms/step
Epoch 110/1000
4/4 - 0s - loss: 0.1538 - accuracy: 0.9909 - 16ms/epoch - 4ms/step
Epoch 111/1000
4/4 - 0s - loss: 0.1468 - accuracy: 0.9939 - 16ms/epoch - 4ms/step
Epoch 112/1000
4/4 - 0s - loss: 0.1409 - accuracy: 0.9939 - 16ms/epoch - 4ms/step
Epoch 113/1000
4/4 - 0s - loss: 0.1328 - accuracy: 0.9879 - 15ms/epoch - 4ms/step
Epoch 114/1000
4/4 - 0s - loss: 0.1304 - accuracy: 0.9939 - 16ms/epoch - 4ms/step
Epoch 115/1000
4

Epoch 203/1000
4/4 - 0s - loss: 9.4629e-04 - accuracy: 1.0000 - 17ms/epoch - 4ms/step
Epoch 204/1000
4/4 - 0s - loss: 8.7560e-04 - accuracy: 1.0000 - 16ms/epoch - 4ms/step
Epoch 205/1000
4/4 - 0s - loss: 9.1578e-04 - accuracy: 1.0000 - 16ms/epoch - 4ms/step
Epoch 206/1000
4/4 - 0s - loss: 0.0021 - accuracy: 1.0000 - 16ms/epoch - 4ms/step
Epoch 207/1000
4/4 - 0s - loss: 0.0028 - accuracy: 1.0000 - 16ms/epoch - 4ms/step
Epoch 208/1000
4/4 - 0s - loss: 0.0013 - accuracy: 1.0000 - 17ms/epoch - 4ms/step
Epoch 209/1000
4/4 - 0s - loss: 7.0567e-04 - accuracy: 1.0000 - 16ms/epoch - 4ms/step
Epoch 210/1000
4/4 - 0s - loss: 6.6140e-04 - accuracy: 1.0000 - 16ms/epoch - 4ms/step
Epoch 211/1000
4/4 - 0s - loss: 6.1045e-04 - accuracy: 1.0000 - 16ms/epoch - 4ms/step
Epoch 212/1000
4/4 - 0s - loss: 5.7465e-04 - accuracy: 1.0000 - 15ms/epoch - 4ms/step
Epoch 213/1000
4/4 - 0s - loss: 5.4832e-04 - accuracy: 1.0000 - 15ms/epoch - 4ms/step
Epoch 214/1000
4/4 - 0s - loss: 5.4347e-04 - accuracy: 1.0000 - 16

Epoch 299/1000
4/4 - 0s - loss: 6.7320e-06 - accuracy: 1.0000 - 16ms/epoch - 4ms/step
Epoch 300/1000
4/4 - 0s - loss: 6.4571e-06 - accuracy: 1.0000 - 15ms/epoch - 4ms/step
Epoch 301/1000
4/4 - 0s - loss: 6.1598e-06 - accuracy: 1.0000 - 15ms/epoch - 4ms/step
Epoch 302/1000
4/4 - 0s - loss: 5.9207e-06 - accuracy: 1.0000 - 16ms/epoch - 4ms/step
Epoch 303/1000
4/4 - 0s - loss: 5.8766e-06 - accuracy: 1.0000 - 15ms/epoch - 4ms/step
Epoch 304/1000
4/4 - 0s - loss: 5.4941e-06 - accuracy: 1.0000 - 16ms/epoch - 4ms/step
Epoch 305/1000
4/4 - 0s - loss: 5.5880e-06 - accuracy: 1.0000 - 16ms/epoch - 4ms/step
Epoch 306/1000
4/4 - 0s - loss: 5.6313e-06 - accuracy: 1.0000 - 16ms/epoch - 4ms/step
Epoch 307/1000
4/4 - 0s - loss: 5.0100e-06 - accuracy: 1.0000 - 16ms/epoch - 4ms/step
Epoch 308/1000
4/4 - 0s - loss: 4.6679e-06 - accuracy: 1.0000 - 16ms/epoch - 4ms/step
Epoch 309/1000
4/4 - 0s - loss: 4.5379e-06 - accuracy: 1.0000 - 16ms/epoch - 4ms/step
Epoch 310/1000
4/4 - 0s - loss: 4.3663e-06 - accuracy:

Epoch 395/1000
4/4 - 0s - loss: 8.2363e-07 - accuracy: 1.0000 - 16ms/epoch - 4ms/step
Epoch 396/1000
4/4 - 0s - loss: 8.1532e-07 - accuracy: 1.0000 - 16ms/epoch - 4ms/step
Epoch 397/1000
4/4 - 0s - loss: 8.0701e-07 - accuracy: 1.0000 - 16ms/epoch - 4ms/step
Epoch 398/1000
4/4 - 0s - loss: 7.9473e-07 - accuracy: 1.0000 - 16ms/epoch - 4ms/step
Epoch 399/1000
4/4 - 0s - loss: 7.8895e-07 - accuracy: 1.0000 - 16ms/epoch - 4ms/step
Epoch 400/1000
4/4 - 0s - loss: 7.7992e-07 - accuracy: 1.0000 - 16ms/epoch - 4ms/step
Epoch 401/1000
4/4 - 0s - loss: 7.7197e-07 - accuracy: 1.0000 - 16ms/epoch - 4ms/step
Epoch 402/1000
4/4 - 0s - loss: 7.6438e-07 - accuracy: 1.0000 - 16ms/epoch - 4ms/step
Epoch 403/1000
4/4 - 0s - loss: 7.5246e-07 - accuracy: 1.0000 - 16ms/epoch - 4ms/step
Epoch 404/1000
4/4 - 0s - loss: 7.4524e-07 - accuracy: 1.0000 - 16ms/epoch - 4ms/step
Epoch 405/1000
4/4 - 0s - loss: 7.2645e-07 - accuracy: 1.0000 - 15ms/epoch - 4ms/step
Epoch 406/1000
4/4 - 0s - loss: 7.2754e-07 - accuracy:

...

Epoch 587/1000
4/4 - 0s - loss: 2.4095e-07 - accuracy: 1.0000 - 15ms/epoch - 4ms/step
Epoch 588/1000
4/4 - 0s - loss: 2.3878e-07 - accuracy: 1.0000 - 15ms/epoch - 4ms/step
Epoch 589/1000
4/4 - 0s - loss: 2.3517e-07 - accuracy: 1.0000 - 16ms/epoch - 4ms/step
Epoch 590/1000
4/4 - 0s - loss: 2.3589e-07 - accuracy: 1.0000 - 16ms/epoch - 4ms/step
Epoch 591/1000
4/4 - 0s - loss: 2.3336e-07 - accuracy: 1.0000 - 15ms/epoch - 4ms/step
Epoch 592/1000
4/4 - 0s - loss: 2.3553e-07 - accuracy: 1.0000 - 15ms/epoch - 4ms/step
Epoch 593/1000
4/4 - 0s - loss: 2.3517e-07 - accuracy: 1.0000 - 16ms/epoch - 4ms/step
Epoch 594/1000
4/4 - 0s - loss: 2.3300e-07 - accuracy: 1.0000 - 15ms/epoch - 4ms/step
Epoch 595/1000
4/4 - 0s - loss: 2.3083e-07 - accuracy: 1.0000 - 15ms/epoch - 4ms/step
Epoch 596/1000
4/4 - 0s - loss: 2.3408e-07 - accuracy: 1.0000 - 15ms/epoch - 4ms/step
Epoch 597/1000
4/4 - 0s - loss: 2.3300e-07 - accuracy: 1.0000 - 15ms/epoch - 4ms/step
Epoch 598/1000
4/4 - 0s - loss: 2.2939e-07 - accuracy:

Epoch 683/1000
4/4 - 0s - loss: 1.7556e-07 - accuracy: 1.0000 - 16ms/epoch - 4ms/step
Epoch 684/1000
4/4 - 0s - loss: 1.7737e-07 - accuracy: 1.0000 - 15ms/epoch - 4ms/step
Epoch 685/1000
4/4 - 0s - loss: 1.7773e-07 - accuracy: 1.0000 - 16ms/epoch - 4ms/step
Epoch 686/1000
4/4 - 0s - loss: 1.7592e-07 - accuracy: 1.0000 - 16ms/epoch - 4ms/step
Epoch 687/1000
4/4 - 0s - loss: 1.7520e-07 - accuracy: 1.0000 - 15ms/epoch - 4ms/step
Epoch 688/1000
4/4 - 0s - loss: 1.7340e-07 - accuracy: 1.0000 - 14ms/epoch - 4ms/step
Epoch 689/1000
4/4 - 0s - loss: 1.7665e-07 - accuracy: 1.0000 - 16ms/epoch - 4ms/step
Epoch 690/1000
4/4 - 0s - loss: 1.7303e-07 - accuracy: 1.0000 - 15ms/epoch - 4ms/step
Epoch 691/1000
4/4 - 0s - loss: 1.7484e-07 - accuracy: 1.0000 - 15ms/epoch - 4ms/step
Epoch 692/1000
4/4 - 0s - loss: 1.7556e-07 - accuracy: 1.0000 - 15ms/epoch - 4ms/step
Epoch 693/1000
4/4 - 0s - loss: 1.7773e-07 - accuracy: 1.0000 - 16ms/epoch - 4ms/step
Epoch 694/1000
4/4 - 0s - loss: 1.7520e-07 - accuracy:

Epoch 779/1000
4/4 - 0s - loss: 1.4016e-07 - accuracy: 1.0000 - 15ms/epoch - 4ms/step
Epoch 780/1000
4/4 - 0s - loss: 1.3944e-07 - accuracy: 1.0000 - 16ms/epoch - 4ms/step
Epoch 781/1000
4/4 - 0s - loss: 1.3799e-07 - accuracy: 1.0000 - 15ms/epoch - 4ms/step
Epoch 782/1000
4/4 - 0s - loss: 1.3836e-07 - accuracy: 1.0000 - 15ms/epoch - 4ms/step
Epoch 783/1000
4/4 - 0s - loss: 1.3872e-07 - accuracy: 1.0000 - 15ms/epoch - 4ms/step
Epoch 784/1000
4/4 - 0s - loss: 1.4016e-07 - accuracy: 1.0000 - 15ms/epoch - 4ms/step
Epoch 785/1000
4/4 - 0s - loss: 1.3872e-07 - accuracy: 1.0000 - 15ms/epoch - 4ms/step
Epoch 786/1000
4/4 - 0s - loss: 1.3619e-07 - accuracy: 1.0000 - 15ms/epoch - 4ms/step
Epoch 787/1000
4/4 - 0s - loss: 1.3727e-07 - accuracy: 1.0000 - 15ms/epoch - 4ms/step
Epoch 788/1000
4/4 - 0s - loss: 1.3872e-07 - accuracy: 1.0000 - 16ms/epoch - 4ms/step
Epoch 789/1000
4/4 - 0s - loss: 1.3980e-07 - accuracy: 1.0000 - 15ms/epoch - 4ms/step
Epoch 790/1000
4/4 - 0s - loss: 1.3799e-07 - accuracy:

Epoch 875/1000
4/4 - 0s - loss: 1.1596e-07 - accuracy: 1.0000 - 15ms/epoch - 4ms/step
Epoch 876/1000
4/4 - 0s - loss: 1.1379e-07 - accuracy: 1.0000 - 15ms/epoch - 4ms/step
Epoch 877/1000
4/4 - 0s - loss: 1.1632e-07 - accuracy: 1.0000 - 16ms/epoch - 4ms/step
Epoch 878/1000
4/4 - 0s - loss: 1.1632e-07 - accuracy: 1.0000 - 15ms/epoch - 4ms/step
Epoch 879/1000
4/4 - 0s - loss: 1.1451e-07 - accuracy: 1.0000 - 15ms/epoch - 4ms/step
Epoch 880/1000
4/4 - 0s - loss: 1.1560e-07 - accuracy: 1.0000 - 16ms/epoch - 4ms/step
Epoch 881/1000
4/4 - 0s - loss: 1.1596e-07 - accuracy: 1.0000 - 15ms/epoch - 4ms/step
Epoch 882/1000
4/4 - 0s - loss: 1.1560e-07 - accuracy: 1.0000 - 16ms/epoch - 4ms/step
Epoch 883/1000
4/4 - 0s - loss: 1.1596e-07 - accuracy: 1.0000 - 15ms/epoch - 4ms/step
Epoch 884/1000
4/4 - 0s - loss: 1.1343e-07 - accuracy: 1.0000 - 16ms/epoch - 4ms/step
Epoch 885/1000
4/4 - 0s - loss: 1.1126e-07 - accuracy: 1.0000 - 15ms/epoch - 4ms/step
Epoch 886/1000
4/4 - 0s - loss: 1.1271e-07 - accuracy:

Epoch 971/1000
4/4 - 0s - loss: 1.0042e-07 - accuracy: 1.0000 - 15ms/epoch - 4ms/step
Epoch 972/1000
4/4 - 0s - loss: 1.0115e-07 - accuracy: 1.0000 - 15ms/epoch - 4ms/step
Epoch 973/1000
4/4 - 0s - loss: 9.7535e-08 - accuracy: 1.0000 - 15ms/epoch - 4ms/step
Epoch 974/1000
4/4 - 0s - loss: 9.9702e-08 - accuracy: 1.0000 - 15ms/epoch - 4ms/step
Epoch 975/1000
4/4 - 0s - loss: 9.8619e-08 - accuracy: 1.0000 - 15ms/epoch - 4ms/step
Epoch 976/1000
4/4 - 0s - loss: 1.0006e-07 - accuracy: 1.0000 - 15ms/epoch - 4ms/step
Epoch 977/1000
4/4 - 0s - loss: 9.8619e-08 - accuracy: 1.0000 - 14ms/epoch - 4ms/step
Epoch 978/1000
4/4 - 0s - loss: 1.0079e-07 - accuracy: 1.0000 - 15ms/epoch - 4ms/step
Epoch 979/1000
4/4 - 0s - loss: 9.9702e-08 - accuracy: 1.0000 - 15ms/epoch - 4ms/step
Epoch 980/1000
4/4 - 0s - loss: 9.8619e-08 - accuracy: 1.0000 - 17ms/epoch - 4ms/step
Epoch 981/1000
4/4 - 0s - loss: 9.6451e-08 - accuracy: 1.0000 - 17ms/epoch - 4ms/step
Epoch 982/1000
4/4 - 0s - loss: 9.5729e-08 - accuracy:

In [25]:
model.save('data/seq2seq.h5')

In [26]:
from tensorflow.keras.models import load_model
model = load_model('data/seq2seq.h5')

In [27]:
# 쉬운 문제
easy_test = [['find', 'PP'],
             ['cold', 'PP'],
             ['cook', 'PP'],
             ['date', 'PP'],
             ['desk', 'PP']]
enc_in, dec_in, dec_out = encoding(easy_test)
enc_in.shape, dec_in.shape

((5, 4, 171), (5, 3, 171))

In [28]:

pred = model.predict([enc_in, dec_in])
pred.shape

1/1 [==============================] - 0s 252ms/step


(5, 3, 171)

In [29]:

#pred[0]
for i in range(len(pred)):
    eng = easy_test[i][0]
    hat = pred[i].argmax(axis=-1)[:-1]
    # kor = alpha[hat[0]] + alpha[hat[1]]
    kor = ''.join([alpha[num] for num in hat])
    print("{} => {}{}".format(eng, kor, hat))

find => 찾다[148  60]
cold => 감기[32 46]
cook => 요리[122  72]
date => 날짜[ 49 146]
desk => 책상[149  97]


In [30]:
# 어려운 문제
hard_test = [['love', 'PP'],
             ['lvoe', 'PP'],
             ['loev', 'PP'],
             ['olve', 'PP'],
             ['loev', 'PP'],
             ['elov', 'PP'],
             ['evol', 'PP'],
             ['rave', 'PP'],]
enc_in, dec_in, _ = encoding(hard_test)
enc_in.shape, dec_in.shape

((8, 4, 171), (8, 3, 171))

In [31]:

pred = model.predict([enc_in, dec_in], verbose=0).argmax(axis=-1)
pred

array([[ 96,  66,   1],
       [ 96,  66,   1],
       [ 96,  66,   1],
       [ 96,  66,   1],
       [ 96,  66,   1],
       [121, 147,   1],
       [ 75,  96,   1],
       [167,  98,   1]], dtype=int64)

In [32]:

for i in range(len(pred)):
    eng = hard_test[i][0]
    kor = ''.join([alpha[num] for num in pred[i]])
    print("{}=>{}{}".format(eng, kor[:-1], pred[i][:-1]))

love=>사랑[96 66]
lvoe=>사랑[96 66]
loev=>사랑[96 66]
olve=>사랑[96 66]
loev=>사랑[96 66]
elov=>왼쪽[121 147]
evol=>매사[75 96]
rave=>회색[167  98]
